# LangChain to prompt programmatically in Python

1. Create an account on [OpenAI Platform](https://platform.openai.com/)
2. Create a project
3. Create an API Key in the project (Settings > Project > API Keys)
4. Paste it into the `.env` file under the variable `OPENAI_API_KEY`

## Load model

In [1]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="chatgpt-4o-latest")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7e1ae08bc290>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7e1ae0827c20>, root_client=<openai.OpenAI object at 0x7e1ae1050b60>, root_async_client=<openai.AsyncOpenAI object at 0x7e1ae0825790>, model_name='chatgpt-4o-latest', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Prompting seralization

### Load markdown template

In [2]:
with open('../prompts/single-day/ES.md', 'r') as file:
    template = file.read()
    
template

'El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.'

### Create prompt object

In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    template_format='f-string',
)

prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')

### Save prompt to JSON

In [4]:
path = '../prompts/single-day/ES.json'
prompt.save(path)

## Chaining with LLMs

### Define chain

In [5]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7e1ae08bc290>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7e1ae0827c20>, root_client=<openai.OpenAI object at 0x7e1ae1050b60>, root_async_client=<openai.AsyncOpenAI object at 0

### Preprocess input data

In [6]:
ticker = 'AAPL'
freq = 'D'

In [7]:
from modules import utils

r = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=1
)

r

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Date
2024-06-11    7.264904
Name: AAPL, dtype: float64

In [8]:
date = r.index[0].strftime('%Y-%m-%d')
change = round(r.values[0], 2)

date, change

('2024-06-11', 7.26)

### Invoke chain with input data

In [9]:
output = chain.invoke({
    'DATE': date,
    'RETURN': change,
    'TICKER': ticker,
    'FREQUENCY': freq
})

print(output)

El movimiento extremo del 7.26% en el precio de las acciones de Apple Inc. (ticker: AAPL) el día 11 de junio de 2024 puede atribuirse principalmente al entusiasmo del mercado tras los anuncios realizados en la Conferencia Mundial de Desarrolladores (WWDC 2024) de Apple, celebrada el 10 de junio de 2024. Durante este evento, Apple presentó una amplia gama de innovaciones centradas en inteligencia artificial (IA), lo que generó una reacción muy positiva entre los inversionistas.

Motivos del movimiento:

1. Integración de inteligencia artificial generativa ("Apple Intelligence"):
Apple anunció una renovada estrategia de inteligencia artificial llamada Apple Intelligence, que incorpora capacidades generativas directamente en su ecosistema de dispositivos. Esta iniciativa incluye mejoras significativas a su asistente de voz Siri, aplicaciones como Mail, Notas y Fotos, y una asociación destacada con OpenAI para integrar ChatGPT en iOS 18, macOS Sequoia y iPadOS 18.

Estas actualizaciones po

## Export output to file

In [10]:
with open(f'reports/{ticker}.md', 'w') as file:
    file.write(output)

---

## Change model to enable web search

### Compose chain

In [11]:
model = ChatOpenAI(model="gpt-4o-search-preview")
chain = prompt | model | StrOutputParser()

chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7e1ad5e7aa50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7e1ad5cb8650>, root_client=<openai.OpenAI object at 0x7e1ad5e39760>, root_async_client=<openai.AsyncOpenAI object at 0

### Invoke chain

In [12]:
output = chain.invoke({
    'DATE': date,
    'RETURN': change,
    'TICKER': ticker,
    'FREQUENCY': freq
})

print(output)

El 10 de junio de 2024, durante la Conferencia Mundial de Desarrolladores (WWDC), Apple presentó varias innovaciones significativas que podrían haber influido en el notable incremento del 7.26% en el precio de sus acciones el 11 de junio de 2024.

Entre los anuncios más destacados se encuentra la introducción de "Apple Intelligence", una plataforma de inteligencia artificial integrada en iOS 18, iPadOS 18 y macOS Sequoia. Esta plataforma combina procesamiento en el dispositivo y en la nube, ofreciendo a los usuarios una experiencia más personalizada y eficiente. La presentación de esta tecnología marcó un paso importante para Apple en el ámbito de la inteligencia artificial. ([es.wikipedia.org](https://es.wikipedia.org/wiki/Apple_Intelligence?utm_source=openai))

Además, Apple anunció la expansión de su dispositivo de realidad mixta, el Apple Vision Pro, a nuevos mercados internacionales. A partir del 28 de junio de 2024, el Apple Vision Pro estaría disponible en países como China, Hon

### Export output to file

In [13]:
with open(f'reports/{ticker}_search.md', 'w') as file:
    file.write(output)